In [ ]:
import wandb
import torchvision
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image
from numpy.linalg import norm
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms
import math
#GNN stuff
import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import umap
print(device)


c:\Users\aidan\Desktop\PRIMES-GNN\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


Metrics


In [ ]:
import numpy as np
from sklearn.metrics.cluster import contingency_matrix

#taken from the google research DMoN paper
def precision(y_true, y_pred):
  true_positives, false_positives, _, _ = _compute_counts(y_true, y_pred)
  return true_positives / (true_positives + false_positives)


def recall(y_true, y_pred):
  true_positives, _, false_negatives, _ = _compute_counts(y_true, y_pred)
  return true_positives / (true_positives + false_negatives)


def accuracy_score(y_true, y_pred):
  true_positives, false_positives, false_negatives, true_negatives = _compute_counts(
      y_true, y_pred)
  return (true_positives + true_negatives) / (
      true_positives + false_positives + false_negatives + true_negatives)


def _compute_counts(y_true, y_pred):  
  contingency = contingency_matrix(y_true, y_pred)
  same_class_true = np.max(contingency, 1)
  same_class_pred = np.max(contingency, 0)
  diff_class_true = contingency.sum(axis=1) - same_class_true
  diff_class_pred = contingency.sum(axis=0) - same_class_pred
  total = contingency.sum()

  true_positives = (same_class_true * (same_class_true - 1)).sum()
  false_positives = (diff_class_true * same_class_true * 2).sum()
  false_negatives = (diff_class_pred * same_class_pred * 2).sum()
  true_negatives = total * (
      total - 1) - true_positives - false_positives - false_negatives

  return true_positives, false_positives, false_negatives, true_negatives



Load High dimen MNIST

In [ ]:
dataset = np.load('HighDimenData/mnist.npy')
file_path = 'HighDimenData/mnist.gt'
labels = []
with open(file_path, 'r') as file:
    for line in file:
        labels.append(int(line.strip()))

#labels = np.load('/notebooks/SVHNdinollabels.npy')
labels= torch.tensor(labels)
dataset = torch.from_numpy(dataset)
print(dataset.size())

torch.Size([70000, 784])


Basic Kmeans block to use with either the DINO dataset or GNN embedded dataset

In [ ]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.cluster import KMeans
import time
start = time.time()

print(time.time()-start)
dataset = dataset.reshape(dataset.shape[0], -1)
model= KMeans(n_clusters= 101, random_state=0, n_init="auto").fit(dataset)
hdbscan_labels = model.fit_predict(dataset)
print(time.time()-start)
print(adjusted_rand_score(labels, hdbscan_labels))
print(adjusted_mutual_info_score(labels, hdbscan_labels))
print(accuracy_score(labels, hdbscan_labels))

Neighbor graph gen

In [4]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
#5
k = 50
nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(dataset)
distances, indices = nbrs.kneighbors(dataset)

# Calculate inverse distances, with a small epsilon to avoid division by zero
epsilon = 1e-5
inverse_distances = 1 / (distances.flatten() + epsilon)

num_nodes = indices.shape[0]
source_nodes = np.repeat(np.arange(num_nodes), k)
target_nodes = indices.flatten()
edge_index = np.vstack([source_nodes, target_nodes])
np.save('edgeindex30k.npy',edge_index)
edge_index = torch.from_numpy(edge_index).long()
edge_weight = torch.from_numpy(inverse_distances).float()

In [5]:
train_mask = np.zeros(num_nodes, dtype=bool)
train_mask[:70000] = True

# Creating the test mask
test_mask = np.zeros(num_nodes, dtype=bool)
test_mask[20000:] = True

train_mask = torch.from_numpy(train_mask)
test_mask = torch.from_numpy(test_mask)

UMAP Dimensionality Reduction

In [ ]:
dataset = umap.UMAP(
    n_neighbors=50,
    min_dist=0.0,
    n_components=512,
    random_state=42,
).fit_transform(dataset)
#np.save('umapmnist.npy', dataset)
#np.save('umapmnistlabels.npy', labels)

c:\Users\aidan\Desktop\PRIMES-GNN\venv\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [6]:
from torch_geometric.loader import NeighborLoader
from torch_geometric.loader import RandomNodeLoader
from torch_geometric.data import Data
data = Data(x=dataset, edge_index=edge_index, edge_weight = edge_weight, y = labels, train_mask = train_mask, test_mask = test_mask)
#30
train_loader = NeighborLoader(data, input_nodes = data.train_mask, num_neighbors=[30]*2, shuffle=True, num_workers = 2, batch_size =64)
test_loader = NeighborLoader(data,input_nodes= data.test_mask, num_neighbors=[-1], shuffle=False, num_workers = 2,batch_size = 32)
sampled_hetero_data = next(iter(train_loader))
print(sampled_hetero_data.y)

tensor([4, 0, 1,  ..., 1, 1, 1])


In [11]:
import torch
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool, GCNConv
from torch_geometric.data import Batch
from torch_geometric.nn import DenseGraphConv, DMoNPooling, GCNConv
from torch_geometric.utils import to_dense_adj

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout_rate=0.5):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        self.dropout_rate = dropout_rate
        self.pool = DMoNPooling([hidden_channels, hidden_channels], out_channels)
        # Initial GAT layer
        self.convs.append(GCNConv(in_channels, hidden_channels, add_self_loops=True))
        self.batch_norms.append(BatchNorm1d(hidden_channels))

        # Hidden GAT layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels, add_self_loops=True))
            self.batch_norms.append(BatchNorm1d(hidden_channels))

        # Final GAT layer
        self.convs.append(GCNConv(hidden_channels, out_channels, add_self_loops=True))
        self.batch_norms.append(BatchNorm1d(out_channels))

    def forward(self, x, edge_index):
        for conv, batch_norm in zip(self.convs[:-1], self.batch_norms[:-1]):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)
            #print(x.size())

       # x = self.convs[-1](x, edge_index, edge_attr = edge_attr)
        adj = to_dense_adj(edge_index)
        #print(x.size())
        x, _, adj, sp1, o1, c1 = self.pool(x, adj)
        #print(x.size())

        return F.log_softmax(x, dim=-1), sp1 + o1 + c1

In [ ]:
import torch
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.data import Batch
from torch_geometric.nn import DenseGraphConv, DMoNPooling, GCNConv
from torch_geometric.utils import to_dense_adj

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout_rate=0.5):
        super(GAT, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        self.dropout_rate = dropout_rate
        self.pool = DMoNPooling([hidden_channels, hidden_channels], out_channels)
        # Initial GAT layer
        self.convs.append(GATConv(in_channels, hidden_channels, add_self_loops=True))
        self.batch_norms.append(BatchNorm1d(hidden_channels))

        # Hidden GAT layers
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels, hidden_channels, add_self_loops=True))
            self.batch_norms.append(BatchNorm1d(hidden_channels))

        # Final GAT layer
        self.convs.append(GATConv(hidden_channels, out_channels, add_self_loops=True))
        self.batch_norms.append(BatchNorm1d(out_channels))

    def forward(self, x, edge_index, edge_attr):
        for conv, batch_norm in zip(self.convs[:-1], self.batch_norms[:-1]):
            x = conv(x, edge_index, edge_attr=edge_attr)
            x = batch_norm(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)


       # x = self.convs[-1](x, edge_index, edge_attr = edge_attr)
        adj = to_dense_adj(edge_index)
       # print(x.size())
        x, _, adj, sp1, o1, c1 = self.pool(x, adj)
       # print(x.size())

        return F.log_softmax(x, dim=-1), sp1 + o1 + c1

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import ReduceLROnPlateau
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = GAT(in_channels= 1024, hidden_channels=128, out_channels=10, num_layers=5, dropout_rate=0.3).to(device)
model = GCN(in_channels= 784, hidden_channels=625, out_channels=10, num_layers=5, dropout_rate=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience= 3, factor = 0.75)
#use this for GNN embedding
#model.load_state_dict(torch.load('/notebooks/gcnmodel_weights.pth'))

Optional Code to further embedd data using a pretrained GNN

In [ ]:
from torch_geometric.loader import  NodeLoader
from torch_geometric.sampler import BaseSampler, NeighborSampler
from torch_geometric.data import Data
data = Data(x=dataset, edge_index=edge_index, edge_weight = edge_weight, y = labels).to(device)
model.eval()  # Set the model to evaluation mode
batch_size = 128
sampler = NeighborSampler(data,[0])
node_loader = NodeLoader(data, sampler,batch_size=batch_size, shuffle=False)
# To store the transformed dataset
transformed_data = np.array([], dtype=np.int32).reshape(0,256)
print(np.shape(transformed_data))
with torch.no_grad():  # Disable gradient calculation for inference
    for batch in node_loader:
        batch = batch.to(device)
        inputs = batch.x  # Get the inputs from the batch
        #print(inputs.size())
        outputs = model(batch.x, batch.edge_index, batch.edge_weight)  # Pass the inputs through the model
        #print(outputs.size())
        transformed_data = np.vstack([transformed_data, outputs.cpu()])
        #print(np.shape(transformed_data))


print("Transformed dataset shape:", np.shape( transformed_data))
np.save('GNNtransformed.npy', transformed_data)

In [9]:
import torch
import torch.nn as nn

class lossmodel(nn.Module):
    def __init__(self, n_clusters, dropout_rate = 0):
        super(lossmodel, self).__init__()
        self.n_clusters = n_clusters
        self.dropout_rate = 0

        # Define the layers
        self.transform = nn.Sequential(
            nn.Linear(n_clusters, n_clusters),
            nn.Dropout(dropout_rate)
        )

        # Initialize weights
        nn.init.orthogonal_(self.transform[0].weight)
        nn.init.zeros_(self.transform[0].bias)

    def forward(self, x):
        return self.transform(x)


def calculate_collapse_loss(number_of_nodes, n_clusters, output, lossize):
    model = lossmodel(n_clusters).to(device)
    assignments = F.softmax(model(output), dim=1)
    cluster_sizes = torch.sum(assignments, dim=0)
    collapse_loss = torch.norm(cluster_sizes) / number_of_nodes * torch.sqrt(torch.tensor(float(n_clusters))) - 1
    return collapse_loss*lossize

In [24]:
config = {
    "architecture":"GIN",
    "clusters":10,
    "dataset":"MNIST",
    "epochs":50,
    "hidden_channel":128,
    "layers":5,
    "learning_rate":0.001,
    "loss_function":"ALL",
    "neighbors":50,
}

# Pass the config dictionary when you initialize W&B
run = wandb.init(project="PRIMES-GNN", config=config, name = 'GIN-highbatch')
#e807d0434ea6864dc92d6c99cdcfa073feaca0cc



Neighbor loader with Plateau scheduler

In [16]:
import time
import torch_sparse
from sklearn.metrics.cluster import normalized_mutual_info_score
classes = 10
def train():
    model.train()
    loss_all = 0
    count = 0
    for data in train_loader:
        count = count+1
        optimizer.zero_grad()
        data = data.to(device)
        #out,tot_loss = model(data.x, data.edge_index, data.edge_weight)
        out,tot_loss = model(data.x, data.edge_index)
        out = out.squeeze()
        loss = tot_loss + calculate_collapse_loss(data.num_nodes,classes,out,0.1)
        loss_all += loss.item()
        loss.backward()
        optimizer.step()
    return loss_all/count


@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    loss_all = 0
    nmi = 0
    count = 0
    for data in loader:
        count +=1
        data.to(device)
        #pred,tot_loss = model(data.x, data.edge_index, data.edge_weight)
        pred,tot_loss = model(data.x, data.edge_index)
        pred = pred.squeeze()
        loss = tot_loss+ calculate_collapse_loss(data.num_nodes,classes,pred,0.1)
        loss_all += loss.item()
        correct += accuracy_score(data.y.cpu().detach().numpy(),pred.max(dim=-1)[1].cpu().detach().numpy())
        nmi += normalized_mutual_info_score(data.y.cpu().detach().numpy(),pred.max(dim=-1)[1].cpu().detach().numpy())
    return loss_all/count, correct/count, nmi/count


times = []
for epoch in range(1, 51):
    start = time.time()
    train_loss = train()
    print(time.time()-start)
    _, train_acc,nmi = test(test_loader)
    print(time.time()-start)
    test_loss, test_acc,nmi = test(test_loader)
    #wandb.log({'acc': test_acc, 'loss': test_loss, "nmi": nmi})
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, '
          f'Train Acc: {train_acc:.3f},  '
          f' Test Loss: {test_loss:.3f}, '
          f'Test Acc: {test_acc:.3f}')
    times.append(time.time() - start)
    if nmi > best_nmi:
        torch.save(model.state_dict(), 'gcnmodel_weights.pth')
        best_nmi = nmi
    scheduler.step(test_loss)

print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

KeyboardInterrupt: 

Metrics For eval

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_rand_score
def metrics():
    model.eval()
    avgacc = 0
    avgARI = 0
    loss_all = 0
    avgprec = 0
    avgrec = 0
    count = 0
    avgnmi = 0
    for data in test_loader:
        count = count +1
        data.to(device)
        pred = model(data.x, data.edge_index, data.edge_weight)
        pred = pred.max(dim=-1)[1].cpu().detach().numpy()
        pred = pred.squeeze()
        labelsfin = data.y.cpu().detach().numpy()
        avgacc += accuracy_score(labelsfin,pred)
        avgrec += recall(labelsfin,pred)
        avgprec +=precision(labelsfin,pred)
        avgARI +=adjusted_rand_score(labelsfin,pred)
        avgnmi += normalized_mutual_info_score(labelsfin,pred)
    print(f"acc score:{avgacc/count}")
    print(f"recall score:{avgrec/count}")
    print(f"precision score:{avgprec/count}")
    print(f"F1:{2*avgprec*avgrec/(avgprec+avgrec)/count}")
    print(f"ARI:{avgARI/count}")
    print(f"NMI:{avgnmi/count}")
metrics()
